In [1]:
#Import

#pandas
import pandas as pd
from pandas import Series,DataFrame

#numpy
import numpy as np

In [2]:
#路径
path='C:\Users\Wvv\Datacastle_subsidy'

#导入card_train数据
card_train_name=['id','consume','where','how','time','amount','remainder']
card_train=pd.read_table(path + '/train/card_train.txt',sep=',',header=None,names=card_train_name)

#删除重复行并浏览前几行数据
card_train=card_train.drop_duplicates()
card_train.head()

,id,consume,where,how,time,amount,remainder
0,1006,POS消费,地点551,淋浴,2013/09/01 00:00:32,0.50,124.90
2,1968,POS消费,地点159,淋浴,2013/09/01 00:00:39,0.10,200.14
4,1406,POS消费,地点660,开水,2013/09/01 00:00:40,0.01,374.42
6,1406,POS消费,地点78,其他,2013/09/01 00:00:40,0.60,373.82
8,13554,POS消费,地点6,淋浴,2013/09/01 00:00:57,0.50,522.37


In [10]:
card_train.head()

,id,consume,where,how,time,amount,remainder,POS_recharge,card_recharge,load_tansfer,payment_collection
0,1006,POS消费,地点551,淋浴,2013/09/01 00:00:32,0.50,124.90,1,0,0,0
2,1968,POS消费,地点159,淋浴,2013/09/01 00:00:39,0.10,200.14,1,0,0,0
4,1406,POS消费,地点660,开水,2013/09/01 00:00:40,0.01,374.42,1,0,0,0
6,1406,POS消费,地点78,其他,2013/09/01 00:00:40,0.60,373.82,1,0,0,0
8,13554,POS消费,地点6,淋浴,2013/09/01 00:00:57,0.50,522.37,1,0,0,0


In [11]:
card_train.groupby('consume')['id'].count()

consume
POS消费    11686938
交易冲正           17
卡充值        197019
卡冻结            70
卡挂失         11901
卡支取             3
卡片开户         9553
卡片销户         7734
卡补办          7676
卡解冻            70
卡解挂          4861
圈存补帐            4
圈存转账        74520
换卡           3740
支付领取        45053
更新卡信息       10792
车载消费           32
锁卡流水            1
Name: id, dtype: int64

In [12]:
card_train.groupby('how')['id'].count()

how
其他         6227
图书馆      276252
开水      3593191
教务处       24509
文印中心      82442
校医院        5696
校车       904913
洗衣房      330984
淋浴      1243977
超市       815946
食堂      4396945
Name: id, dtype: int64

In [7]:
card_train2=pd.read_csv('input/card_train_time.csv')

In [8]:
card_train2.head()

,id,consume,where,how,time,amount,remainder,date,month,hour,weekday
0,1006,POS消费,地点551,淋浴,2013-09-01 00:00:32,0.50,124.90,09-01-2013,9,0,Sunday
1,1968,POS消费,地点159,淋浴,2013-09-01 00:00:39,0.10,200.14,09-01-2013,9,0,Sunday
2,1406,POS消费,地点660,开水,2013-09-01 00:00:40,0.01,374.42,09-01-2013,9,0,Sunday
3,1406,POS消费,地点78,其他,2013-09-01 00:00:40,0.60,373.82,09-01-2013,9,0,Sunday
4,13554,POS消费,地点6,淋浴,2013-09-01 00:00:57,0.50,522.37,09-01-2013,9,0,Sunday


In [3]:
#表信息
card_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12089909 entries, 0 to 12455557
Data columns (total 7 columns):
id           int64
consume      object
where        object
how          object
time         object
amount       float64
remainder    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 737.9+ MB


In [4]:
#定义time转化函数

from datetime import datetime
def time_tansform(x):
    return datetime.strptime(x,'%Y/%m/%d %H:%M:%S')

In [5]:
#转换字符串型的time成时间戳格式

card_train['time']=card_train['time'].apply(time_tansform)

In [6]:
#时间格式转换

card_train['date']=card_train['time'].apply(lambda x:x.strftime('%m-%d-%Y'))
card_train['month']=card_train['time'].apply(lambda x:x.strftime('%m'))
card_train['hour']=card_train['time'].apply(lambda x:x.strftime('%H'))
card_train['weekday']=card_train['time'].apply(lambda x:x.strftime('%A'))

In [7]:
#上面数据处理花时很多，先保存处理后的结果

card_train.to_csv('input/card_train_time.csv',index=False)

In [12]:
#hour

#将每天分成8个时段

a=['00','01']#熬夜
b=['06','07']#早餐
c=['08','09']#早餐
d=['11','12','13']#午饭
e=['17','18','19','20','21']#晚饭
f=['22','23']#宵夜
g=['02','03','04','05']#深夜
#h=['10','14','15','16']#其它
def hour_transform(hour):
    if hour in a:
        return 'a'
    elif hour in b:
        return 'b'
    elif hour in c:
        return 'c'
    elif hour in d:
        return 'd'
    elif hour in e:
        return 'e'
    elif hour in f:
        return 'f'
    elif hour in g:
        return 'g'
    else:
        return 'h'
    
card_train['hour']=card_train['hour'].apply(hour_transform)

#对hour进行one_hot编码得到hour新表
hour_dummies=pd.get_dummies(card_train['hour'])
hour_dummies.columns = ['hour_00_01','hour_06_07','hour_08_09','hour_11_13','hour_17_21','hour_22_23','hour_02_05','hour_other']

#连接两表
card_train = card_train.join(hour_dummies)

In [14]:
#weekday

#分为周末与非周末
a=['Monday','Tuesday','Wednesday','Thursday','Friday']
#b=['Saturday','Sunday']
def week_tansform(weekday):
    if weekday in a:
        return 'wd'
    else:
        return 'vd'
    
card_train['weekday']=card_train['weekday'].apply(week_tansform)

#对weekday进行one_hot编码得到weekday新表
weekday_dummies = pd.get_dummies(card_train['weekday'])
weekday_dummies.columns=['resting_day','working_day']
card_train=card_train.join(weekday_dummies)

In [9]:
#consume

#对consume进行one_hot编码得到consume新表
consume_dummies = pd.get_dummies(card_train['consume'])

#只保留使用数量多且关于消费的特称
consume_dummies.drop(['交易冲正','卡冻结','卡挂失','卡支取','卡片开户','卡片销户','卡补办',
                      '卡解冻','卡解挂','圈存补帐','换卡','更新卡信息','车载消费','锁卡流水'], axis=1, inplace=True)
consume_dummies.columns = ['POS_recharge','card_recharge','load_tansfer','payment_collection']

#连接表
card_train = card_train.join(consume_dummies)

In [23]:
#how

#对consume进行one_hot编码得到consume新表
how_dummies = pd.get_dummies(card_train['how'])
how_dummies.columns=['other','library','water','office','printting','hospital','bus','landry','shower','market','canteen']
card_train = card_train.join(how_dummies)

In [24]:
#获得前十大最受欢迎的食堂

card_train_canteen=card_train.loc[card_train['canteen']==1]
card_train_canteen=DataFrame(card_train_canteen.groupby('where')['where'].count())
card_train_canteen=card_train_canteen.sort(['where'],ascending=False)
card_train_canteen[:10]

C:\Users\Wvv\Anaconda3\envs\py2\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,where
where,
地点232,377852
地点118,154008
地点83,112593
地点61,87774
地点1155,85891
地点217,79599
地点69,76509
地点72,73761
地点60,69832


In [25]:
#获得前十大最受欢迎的非食堂地点

card_train_where=card_train.loc[card_train['canteen']==0]
card_train_where=DataFrame(card_train_where.groupby('where')['where'].count())
card_train_where=card_train_where.sort(['where'],ascending=False)
card_train_where[:10]

C:\Users\Wvv\Anaconda3\envs\py2\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,where
where,
地点21,2222671
地点91,655846
地点188,178516
地点192,169287
地点26,138544
地点996,121796
地点6,113786
地点219,101091
地点190,92155


In [26]:
#where

a=['地点232','地点118','地点83','地点61','地点1155','地点217','地点69','地点72','地点60','地点841']
b=list(card_train_canteen.index)
c=['地点21','地点91','地点188','地点192','地点26','地点996','地点6','地点219','地点190','地点248']
#d=list(card_test_where.index)
def where_tansform(where):
    if where in a:
        return 'a'
    elif where not in a and where in b:
        return 'b'
    elif where in c:
        return 'c'
    else:
        return 'd'
    
card_train['where']=card_train['where'].apply(where_tansform)

#对where进行one_hot编码得到where新表

where_dummies = pd.get_dummies(card_train['where'])
where_dummies.columns = ['f_ten_canteen','other_canteen','f_ten_place','other_place']
card_train = card_train.join(where_dummies)

In [30]:
#month

#分为在校，寒假与暑假

a=['01','02']#寒假
b=['07','08']#暑假
def month_transform(month):
    if month in a:
        return 'a'
    elif month in b:
        return 'b'
    else:
        return 'c'
    
card_train['month']=card_train['month'].apply(month_transform)

#对month进行one_hot编码得到where新表

month_dummies = pd.get_dummies(card_train['month'])
month_dummies.columns=['winter_vacation','summer_vacation','on_school']
card_train=card_train.join(month_dummies)

In [33]:
#删除无用的列

card_train.drop(['consume','where','how','time','hour','weekday','month'],axis=1,inplace=True)

In [35]:
#导出数据

card_train.to_csv('input/card_train.csv',index=False)

In [34]:
card_train.head()

,id,amount,remainder,date,hour_00_01,hour_06_07,hour_08_09,hour_11_13,hour_17_21,hour_22_23,...,shower,market,canteen,f_ten_canteen,other_canteen,f_ten_place,other_place,winter_vacation,summer_vacation,on_school
0,1006,0.50,124.90,09-01-2013,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
2,1968,0.10,200.14,09-01-2013,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
4,1406,0.01,374.42,09-01-2013,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
6,1406,0.60,373.82,09-01-2013,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
8,13554,0.50,522.37,09-01-2013,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
